# CSV2RDATA

In [205]:
import pandas as pd
import pyreadr
import glob

In [2]:
def show_enum(arr: []) -> {}:
    return {i:file for i,file in enumerate(arr)}

In [164]:
OBSERVED_DATA = 'observed_data'

In [165]:
paths = glob.glob(f'{OBSERVED_DATA}/**/**/**/*.csv')
show_enum(paths)

{0: 'observed_data/0_RioDePedras_Fluviais/Rio De Pedras Fluviais/cotas_C_46520000/cotas_C_46520000.csv',
 1: 'observed_data/0_RioDePedras_Fluviais/Rio De Pedras Fluviais/PerfilTransversal_C_46520000/PerfilTransversal_C_46520000.csv',
 2: 'observed_data/0_RioDePedras_Fluviais/Rio De Pedras Fluviais/vazoes_C_46520000/vazoes_C_46520000.csv',
 3: 'observed_data/10_Tagua_Fluviais/Tagua Fluviais/cotas_C_46415000/cotas_C_46415000.csv',
 4: 'observed_data/10_Tagua_Fluviais/Tagua Fluviais/cotas_C_46455000/cotas_C_46455000.csv',
 5: 'observed_data/10_Tagua_Fluviais/Tagua Fluviais/cotas_C_46490000/cotas_C_46490000.csv',
 6: 'observed_data/10_Tagua_Fluviais/Tagua Fluviais/cotas_C_46520000/cotas_C_46520000.csv',
 7: 'observed_data/10_Tagua_Fluviais/Tagua Fluviais/cotas_C_46543000/cotas_C_46543000.csv',
 8: 'observed_data/10_Tagua_Fluviais/Tagua Fluviais/cotas_C_46550000/cotas_C_46550000.csv',
 9: 'observed_data/10_Tagua_Fluviais/Tagua Fluviais/cotas_C_46570000/cotas_C_46570000.csv',
 10: 'observed_

In [166]:
df = pd.DataFrame()
for path in paths:
    try:
        template = {}

        text = path.split('/')
        value = text[-1].split('_')[0]
        template['source'] = text[-3].split(' ')[-1]
        template['station'] = ' '.join(text[-3].split(' ')[:-1])
        template['agency_cd'] = 'ANA'

        aux = pd.read_csv(path, encoding='latin', sep=';', header=None, decimal=',')

        # Get valid columns and rows
        idx = aux[aux.iloc[:,0] == 'EstacaoCodigo'].index[0]
        aux.columns = aux.iloc[idx]
        aux.drop(range(idx+1), inplace=True)
        aux.dropna(how='all', inplace=True)
        aux.drop(list(aux.filter(regex='Status')), axis=1, inplace=True)

        # Changes date format
        if 'Data' in aux.columns:
            aux['Data'] = pd.to_datetime(aux['Data'], format='%d/%m/%Y')
        if 'Hora' in aux.columns:
            aux['Hora'] = pd.to_datetime(aux['Hora'], format='%d/%m/%Y %H:%M').dt.strftime('%H:%M:%S')

        ts = []
        for i, row in aux.iterrows():
            template['code_station'] = row['EstacaoCodigo']
            template['date'] = row.get('Data')
            template['time'] = row.get('Hora')

            for col in aux.filter(regex='\d'):
                template[value] = str(row[col]).replace(',','.')
                ts.append(template.copy())
                template['date'] += pd.DateOffset(days=1)
        ts = pd.DataFrame(ts)
        ts = ts.set_index('date')
        df = pd.concat([df,ts])
    except Exception as e:
        print(f'[ERROR] {path}: {e}')
df

[ERROR] observed_data/0_RioDePedras_Fluviais/Rio De Pedras Fluviais/PerfilTransversal_C_46520000/PerfilTransversal_C_46520000.csv: Error tokenizing data. C error: Expected 1 fields in line 12, saw 15



,source,station,agency_cd,code_station,time,cotas,vazoes,chuvas
date,,,,,,,,
2019-03-01,Fluviais,Rio De Pedras,ANA,46520000,17:00:00,76,NaN,NaN
2019-03-02,Fluviais,Rio De Pedras,ANA,46520000,17:00:00,76,NaN,NaN
2019-03-03,Fluviais,Rio De Pedras,ANA,46520000,17:00:00,76,NaN,NaN
2019-03-04,Fluviais,Rio De Pedras,ANA,46520000,17:00:00,77,NaN,NaN
2019-03-05,Fluviais,Rio De Pedras,ANA,46520000,17:00:00,79,NaN,NaN
...,...,...,...,...,...,...,...,...
2019-03-27,Fluviais,Sao Sebastiao,ANA,46610000,NaN,NaN,118.325,NaN
2019-03-28,Fluviais,Sao Sebastiao,ANA,46610000,NaN,NaN,96.595,NaN
2019-03-29,Fluviais,Sao Sebastiao,ANA,46610000,NaN,NaN,106.918,NaN


In [167]:
df.to_csv(f'{OBSERVED_DATA}/full.csv')

Conversão para estrutura do RDATA

In [216]:
df = pd.read_csv(f'{OBSERVED_DATA}/full.csv')

In [217]:
station = 'Boqueirao'

In [218]:
df_station = df[df['station'] == station]
df_station

,date,source,station,agency_cd,code_station,time,cotas,vazoes,chuvas
10509,2019-03-01,Fluviais,Boqueirao,ANA,46415000,NaN,70.0,NaN,NaN
10510,2019-03-02,Fluviais,Boqueirao,ANA,46415000,NaN,76.0,NaN,NaN
10511,2019-03-03,Fluviais,Boqueirao,ANA,46415000,NaN,78.0,NaN,NaN
10512,2019-03-04,Fluviais,Boqueirao,ANA,46415000,NaN,78.0,NaN,NaN
10513,2019-03-05,Fluviais,Boqueirao,ANA,46415000,NaN,76.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12426,2019-03-27,Fluviais,Boqueirao,ANA,46902000,NaN,NaN,278.966,NaN
12427,2019-03-28,Fluviais,Boqueirao,ANA,46902000,NaN,NaN,283.549,NaN
12428,2019-03-29,Fluviais,Boqueirao,ANA,46902000,NaN,NaN,290.136,NaN
12429,2019-03-30,Fluviais,Boqueirao,ANA,46902000,NaN,NaN,296.768,NaN


In [219]:
df_station['site_no'] = 7422
df_station

/home/ferraz/.conda/envs/csv2rdata/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,source,station,agency_cd,code_station,time,cotas,vazoes,chuvas,site_no
10509,2019-03-01,Fluviais,Boqueirao,ANA,46415000,NaN,70.0,NaN,NaN,7422
10510,2019-03-02,Fluviais,Boqueirao,ANA,46415000,NaN,76.0,NaN,NaN,7422
10511,2019-03-03,Fluviais,Boqueirao,ANA,46415000,NaN,78.0,NaN,NaN,7422
10512,2019-03-04,Fluviais,Boqueirao,ANA,46415000,NaN,78.0,NaN,NaN,7422
10513,2019-03-05,Fluviais,Boqueirao,ANA,46415000,NaN,76.0,NaN,NaN,7422
...,...,...,...,...,...,...,...,...,...,...
12426,2019-03-27,Fluviais,Boqueirao,ANA,46902000,NaN,NaN,278.966,NaN,7422
12427,2019-03-28,Fluviais,Boqueirao,ANA,46902000,NaN,NaN,283.549,NaN,7422
12428,2019-03-29,Fluviais,Boqueirao,ANA,46902000,NaN,NaN,290.136,NaN,7422
12429,2019-03-30,Fluviais,Boqueirao,ANA,46902000,NaN,NaN,296.768,NaN,7422


In [220]:
cols = ['date', 'agency_cd', 'site_no', 'vazoes']
f1 = df_station['code_station'] == 46415000
f2 = df_station['time'].isnull()
df_filter = df_station.loc[f1&f2, cols]
df_filter

,date,agency_cd,site_no,vazoes
10509,2019-03-01,ANA,7422,NaN
10510,2019-03-02,ANA,7422,NaN
10511,2019-03-03,ANA,7422,NaN
10512,2019-03-04,ANA,7422,NaN
10513,2019-03-05,ANA,7422,NaN
...,...,...,...,...
12023,2019-03-27,ANA,7422,28.445
12024,2019-03-28,ANA,7422,30.520
12025,2019-03-29,ANA,7422,32.301
12026,2019-03-30,ANA,7422,31.583


In [222]:
df_filter.dropna(inplace=True)
df_filter

,date,agency_cd,site_no,vazoes
11997,2019-03-01,ANA,7422,19.678
11998,2019-03-02,ANA,7422,23.249
11999,2019-03-03,ANA,7422,24.502
12000,2019-03-04,ANA,7422,24.502
12001,2019-03-05,ANA,7422,23.249
12002,2019-03-06,ANA,7422,23.249
12003,2019-03-07,ANA,7422,23.249
12004,2019-03-08,ANA,7422,22.028
12005,2019-03-09,ANA,7422,22.028
12006,2019-03-10,ANA,7422,22.635


In [223]:
df_filter = df_filter.rename(columns={'date': 'Date', 'vazoes': 'obs'})
df_filter

,Date,agency_cd,site_no,obs
11997,2019-03-01,ANA,7422,19.678
11998,2019-03-02,ANA,7422,23.249
11999,2019-03-03,ANA,7422,24.502
12000,2019-03-04,ANA,7422,24.502
12001,2019-03-05,ANA,7422,23.249
12002,2019-03-06,ANA,7422,23.249
12003,2019-03-07,ANA,7422,23.249
12004,2019-03-08,ANA,7422,22.028
12005,2019-03-09,ANA,7422,22.028
12006,2019-03-10,ANA,7422,22.635


In [225]:
df_filter.index = range(len(df_filter))

In [226]:
pyreadr.write_rdata("obsStrData.Rdata", df_filter, df_name="obsStrData")